<a href="https://colab.research.google.com/github/abishinjoseph/Encored/blob/main/Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.impute import SimpleImputer

# Load data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Prepare data
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values
X_test = test_data.values

# Handling missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define base classifiers
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(kernel='linear', C=0.1, probability=True, random_state=42))
]

# Define meta-classifier
meta_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Initialize stacking classifier
stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier)

# Train stacking classifier
stacking_clf.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = stacking_clf.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Stacking Classifier Accuracy:", accuracy)

# Make predictions on test data
test_predictions = stacking_clf.predict(X_test)

# Prepare submission file
submission_df = pd.DataFrame({'ImageId': range(1, len(test_predictions) + 1), 'Label': test_predictions})

# Save submission file
submission_df.to_csv("submission.csv", index=False)


Stacking Classifier Accuracy: 0.940406976744186
